In [1]:
from data_prep import *
from PositionModel import *
from PlayerModel import *
from commons import *
import shap
from sklearn.ensemble import RandomForestRegressor

In [2]:
features = ['goals_scored_avg5', 'xG_avg5', 'assists_fpl_avg5',
       'bonus_avg5', 'bps_avg5', 'clean_sheets_avg5', 'goals_conceded_avg5',
       'shots_avg5', 'xGBuildup_avg5', 'xGChain_avg5', 'yellow_cards_avg5',
       'red_cards_avg5', 'was_home', 'rating_diff', 'minutes_avg5']
rolling_columns = ['goals_scored', 'xG', 'assists_fpl', 'bonus', 'bps', 'clean_sheets', 'goals_conceded', 'shots', 'xGBuildup', 'xGChain', 'yellow_cards', 'red_cards', 'minutes']
to_predict = 'total_points'

In [3]:
# loading dataset
previous_games = merged_understat_and_fpl(rolling_columns, True)
# previous_games = load_dataset()

Columns (5,39) have mixed types. Specify dtype option on import or set low_memory=False.


LENGTH (0, 41)


ValueError: Incompatible indexer with Series

In [ ]:
previous_games.columns

In [ ]:
# splitting data
X_train_def = previous_games[previous_games['position_fpl'] == 'DEF'][features]
y_train_def = previous_games[previous_games['position_fpl'] == 'DEF'][to_predict]

X_train_mid = previous_games[previous_games['position_fpl'] == 'MID'][features]
y_train_mid = previous_games[previous_games['position_fpl'] == 'MID'][to_predict]

X_train_fwd = previous_games[previous_games['position_fpl'] == 'FWD'][features]
y_train_fwd = previous_games[previous_games['position_fpl'] == 'FWD'][to_predict]

In [ ]:
# general model fitting
position_model = PositionModel()

In [ ]:
position_model.def_model.fit(X_train_def, y_train_def)
position_model.mid_model.fit(X_train_mid, y_train_mid)
position_model.fwd_model.fit(X_train_fwd, y_train_fwd)

In [ ]:
previous_games[previous_games['player_name'] == 'Mohamed Salah']

In [ ]:
# getting next gameweek
next_gameweek = get_next_gameweek(previous_games, NEXT_GAMEWEEK, rolling_columns)

In [ ]:
# removing NaNs
next_gameweek = next_gameweek.dropna(subset = features)

In [ ]:
next_gameweek_gk = next_gameweek[next_gameweek['position_fpl'] == 'GK']
next_gameweek_def = next_gameweek[next_gameweek['position_fpl'] == 'DEF']
next_gameweek_mid = next_gameweek[next_gameweek['position_fpl'] == 'MID']
next_gameweek_fwd = next_gameweek[next_gameweek['position_fpl'] == 'FWD']

In [ ]:
def get_shap_index_by_name(df, name):
    index = 0
    for i, r in df.iterrows():
        if r['name'] == name:
            return index
        index += 1
    return None

# DEFENDERS

In [ ]:
# predicting
pred_points = position_model.def_model.predict(next_gameweek_def[features])

In [ ]:
predictions = next_gameweek_def
predictions.insert(1, 'pred_points', pred_points)
predictions = predictions.sort_values(by = 'pred_points', ascending = False)

In [ ]:
predictions[['name', 'was_home', 'opp_team_rating', 'pred_points']].head(20)

In [ ]:
# shap for predictions
explainer_pred = shap.Explainer(position_model.def_model, X_train_def)
shap_values_pred = explainer_pred(next_gameweek_def[features], check_additivity=False)

In [ ]:
# feature influence
shap.plots.beeswarm(shap_values_pred)

In [ ]:
# feature importance
shap.plots.bar(shap_values_pred)

# MIDFIELDERS

In [ ]:
# predicting
pred_points = position_model.mid_model.predict(next_gameweek_mid[features])

In [ ]:
predictions = next_gameweek_mid
predictions.insert(1, 'pred_points', pred_points)
predictions = predictions.sort_values(by = 'pred_points', ascending = False)

In [ ]:
predictions[['name', 'was_home', 'opp_team_rating', 'pred_points']].head(20)

In [ ]:
# shap for predictions
explainer_pred = shap.Explainer(position_model.mid_model, X_train_def)
shap_values_pred = explainer_pred(next_gameweek_mid[features], check_additivity=False)

In [ ]:
# shap for previous games
explainer = shap.Explainer(position_model.mid_model, X_train_mid)
shap_values = explainer(X_train_def, check_additivity=False)

In [ ]:
# feature influence
shap.plots.beeswarm(shap_values)

In [ ]:
# feature importance
shap.plots.bar(shap_values)

# FORWARDS

In [ ]:
# predicting
pred_points = position_model.fwd_model.predict(next_gameweek_fwd[features])

In [ ]:
predictions = next_gameweek_fwd
predictions.insert(1, 'pred_points', pred_points)
predictions = predictions.sort_values(by = 'pred_points', ascending = False)

In [ ]:
predictions[['name', 'was_home', 'opp_team_rating', 'pred_points']].head(20)

In [ ]:
# shap for predictions
explainer_pred = shap.Explainer(position_model.mid_model, X_train_def)
shap_values_pred = explainer_pred(next_gameweek_mid[features], check_additivity=False)

In [ ]:
# shap for previous games
explainer = shap.Explainer(position_model.mid_model, X_train_mid)
shap_values = explainer(X_train_def, check_additivity=False)

In [ ]:
# feature influence
shap.plots.beeswarm(shap_values)